In [65]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly
import plotly.graph_objects as go
from dash.dependencies import Input, Output
import dash_table
import pandas as pd
import numpy as np
import os
import folium

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("datatdash\\liensvilles.csv", dtype="unicode")
df_auto = pd.read_csv("datatdash\\auto.csv", dtype="unicode")
df_chomage = pd.read_csv("datatdash\\chomage.csv", dtype="unicode")
df_csp = pd.read_csv("datatdash\\csp.csv", dtype="unicode")
df_del = pd.read_csv("datatdash\\delinquance.csv", dtype="unicode")
df_demo = pd.read_csv("datatdash\\demographie.csv", dtype="unicode")
df_elections = pd.read_csv("datatdash\\elections.csv", dtype="unicode")
df_emploi = pd.read_csv("datatdash\\emploi.csv", dtype="unicode")
df_entreprises = pd.read_csv("datatdash\\entreprises.csv", dtype="unicode")
df_immo = pd.read_csv("datatdash\\immobilier.csv", dtype="unicode")
df_infos = pd.read_csv("datatdash\\infos.csv", dtype="unicode")
df_salaires = pd.read_csv("datatdash\\salaires.csv", dtype="unicode")
df_sante = pd.read_csv("datatdash\\santeSocial.csv", dtype="unicode")


In [4]:
df_infos.columns

Index(['ville', 'lien', 'Code Insee', 'Région', 'Département',
       'Etablissement public de coopération intercommunale (EPCI)',
       'Code postal (CP)', 'Nom des habitants', 'Population (2015)',
       'Population : rang national (2015)', 'Densité de population (2015)',
       'Taux de chômage (2015)', 'Pavillon bleu', 'Ville d'art et d'histoire',
       'Ville fleurie', 'Ville internet', 'Superficie (surface)',
       'Altitude min.', 'Altitude max.', 'Latitude', 'Longitude'],
      dtype='object')

In [5]:
# Dropdown
villes = [{"label": ville, "value": ville} for ville in df["ville"].unique()]

In [ ]:
app = dash.Dash("_name_")

app.layout = html.Div([
    
    html.Div([
        html.H4("Choisissez une ville: "),
        dcc.Dropdown(
            id = "ville-picker",
            options = villes,
            value = " Paris (75000)"
        )], style={
            "width":"25%",
            "border":"1px solid #eee",
            "padding":"30px 30px 30px 120px",
            "box-shadow" : "0 2px 2px #ccc",
            "display" : "inline-block",
            "verticalAlign":"top"
        }),
    
    html.Div([
        dcc.Tabs(id = "tabs", value = "tab-1", children = [
            # infos générales
            dcc.Tab(label = "Infos Générales", children = [
                html.Div([
                    html.H3("Infos Générales")], 
                    style = {"background":"blue", "color":"white", "textAlign":"center", "paddind":"10px 0px 10px, 0px"}),
                html.Div([
                    dash_table.DataTable(
                        id = "table_infos",
                        style_cell = {'font-family':'Montserrat'}
                    )
                ], style = {'width' : '40%', 'border':'1px solid #eee', 'box-shadow':'0 2px #ccc', 'display':'inline-block'}),
                html.Div(id = "map", children = [], style = {'display':'inline-block', 'verticalAlign':'top', 'width':'50%',
                                                            'padding':'15px 0px 15px, 0px'})
            ]),
            
            # Demographies
            dcc.Tab(label = "Demographie", children = [
                html.H3("Population Française", style = {"background":"blue", "color":"white", "textAlign":"center",
                                                        "padding":"10px 0px 10px 0px"}),
                html.Div([
                    dcc.Graph(id = "population")
                ], style = {"border":"1px solid #eee", "box-shadow":"0 2px 2px #ccc", 'display':'inline-block', 'verticalAlign':'top',
                           "width":"45%", "padding":"50px 0px 0px 50px"}),
                html.Div([
                    dcc.Graph(id = "naissances_deces")
                ], style = {"border":"1px solid #eee", "box-shadow":"0 2px 2px #ccc", 'display':'inline-block', 'verticalAlign':'top',
                           "width":"45%", "padding":"50px 0px 0px 50px"})
            ] ),
            
            # Sante et social
            dcc.Tab(label = "Santé et social"),
            
            # Immobilier
            dcc.Tab(label = "Immobilier"),
            
            # Entreprises
            dcc.Tab(label = "Entreprises"),
            
            # Emploi
            dcc.Tab(label = "Emploi"),
            
            # Salaires
            dcc.Tab(label = "Salaires"),
            
            # CSP
            dcc.Tab(label = "CSP"),
            
            # Automobiles
            dcc.Tab(label = "Automobiles"),
            
            # Delinquance
            dcc.Tab(label = "Delinquance"),
            
            # Européennes
            dcc.Tab(label = "Européennes 2019"),
        ])
    ])
])

@app.callback([Output("table_infos", "data"), Output("table_infos", "columns")], [Input("ville-picker", "value")])
def update_generales(ville_choisie):
    colonnes = df_infos.columns
    colonnes_offs = ["Taux de chômage (2015)", "Etablissement public de coopération intercommunale (EPCI)",
                    'lien', "Unnamed: 0", 'Pavillon bleu', "Ville d'art et d'histoire", "Ville fleurie", 
                     'Ville internet', 'ville']
    listeInfos = [info for info in colonnes if info not in colonnes_offs]
    infos = {
        "intitule" : listeInfos,
        "donnee" : [df_infos[df_infos["ville"] == ville_choisie][col].iloc[0] for col in listeInfos]
    }
    table = pd.DataFrame(infos)
    data = table.to_dict("records")
    entete = {"id":"intitule", "name":"  "}, {"id":"donnee", "name": ville_choisie}
    return data, entete

@app.callback(Output('map', 'children'), [Input('ville-picker', 'value')])
def update_location(ville_choisie):
    longitude = df_infos[df_infos['ville'] == ville_choisie]['Longitude'].iloc[0]
    latitude = df_infos[df_infos['ville'] == ville_choisie]['Latitude'].iloc[0]
    
    carte = folium.Map(location = (latitude, longitude), zoom_start=6)
    marker = folium.Marker(location = [latitude, longitude])
    
    marker.add_to(carte)
    
    fichier = "locations\\localisation_" + ville_choisie + ".html"
    
    if not os.path.isfile(fichier):
        carte.save(fichier)
    
    return html.Iframe(srcDoc = open(fichier, "r").read(), width = '100%', height = '600')

#DEMOGRAPHIE
@app.callback(Output("population", "figure"), [Input("ville-picker", "value")])
def population_graph(ville_choisie):
    x_axis = np.array(range(2006, 2016))
    y_axis = [
        df_demo[df_demo['ville'] == ville_choisie]["nbre habitants (" + str(annee) + ")"].iloc[0] for annee in range(2006, 2016)
    ]
    
    traces = []
    traces.append(
        go.Scatter(x = x_axis, y = y_axis, mode = "lines+markers", line = {"shape" : "spline", "smoothing":1})
    )
    
    return {
        "data": traces,
        "layout":go.Layout(
            title = "Evolution de la population a " + ville_choisie,
            xaxis = {'title' : 'Annees'},
            yaxis = dict(title = "Nombre d'habitants"),
            hovermode = 'closest',
            legend_orientation = 'h'
        )
    }

@app.callback(Output("naissances_deces", "figure"), [Input("ville-picker", "value")])
def naissances_deces_graph(ville_choisie):
    x_axis = np.array(range(1999, 2017))
    y_axis_naissances = [
        df_demo[df_demo['ville'] == ville_choisie]["nbre naissances (" + str(a) + ")"].iloc[0] for a in range(1999, 2017)
    ]
    
    y_axis_deces = [
        df_demo[df_demo['ville'] == ville_choisie]["nbre deces (" + str(a) + ")"].iloc[0] for a in range(1999, 2017)
    ]
    
    traces = [
        go.Scatter(x = x_axis, y = y_axis_naissances, mode = "lines+markers", line = {"shape" : "spline", "smoothing":1}),
        go.Scatter(x = x_axis, y = y_axis_deces, mode = "lines+markers", line = {"shape" : "spline", "smoothing":1})
    ]
    
    
    return {
        "data": traces,
        "layout":go.Layout(
            title = "Evolution des naissances et deces à " + ville_choisie,
            xaxis = {'title' : 'Annees'},
            yaxis = dict(title = "Nombre de personnes"),
            hovermode = 'closest',
            legend_orientation = 'h'
        )
    }


#
    

serveur = app.server
app.run(debug = False)